## Cell Fate Decision Generative Model

The form of our data: 

After processing, we obtain number density fractions of difference cell types. These are as follows:

$n_b$=number density of pluripotent cells (both markers) per unit area (# 'both' cells in box)

$n_r$=density of red cells (Oct4) (# 'red' cells in box)

$n_g$=density of green cells (Sox2) (# 'green' cells in box)

Between two time steps, we assert the following: 

$$\Delta n_b= n_{entering, b} - n_{leaving, b} + n_{dividing, b} -n_{b\rightarrow g } - n_{g\rightarrow b}$$

Where $n_{entering, b}$ denotes the number of pluripotent cells entering the ROI, $n_{leaving, b}$ denotes the number exiting the ROI, $n_{dividing,b}$ denotes the number of pluripotent cells in the ROI that divide, $n_{b\rightarrow g } $ the number of cells that differentiate to green, and $n_{g\rightarrow b}$ the number of cells that differentiate to blue. 

$$\Delta n_g= n_{entering, g} - n_{leaving, g} + n_{dividing, g} +n_{b\rightarrow g } $$

Where $n_{entering, g}$ denotes the number of green cells entering the ROI, $n_{leaving, g}$ denotes the number exiting the ROI, and $n_{dividing,g}$ denotes the number of green cells in the ROI that divide.  

$$\Delta n_r= n_{entering, r} - n_{leaving, r} + n_{dividing, r} +n_{b\rightarrow r } $$

With variables defined as above. 

We can then convert these to differential equations (modulo some scaling factor):


$$ \frac{d n_b (t,x)}{dt}  = k_{entry}n_b(t,x_{neighbors})+(k_{division}-k_{departure}) n_b(t,x) - (k_{b\rightarrow g } + k_{b \rightarrow r})n_{b}(t,x) $$

$$ \frac{d n_g (t,x)}{dt} = k_{entry}n_g(t,x_{neighbors})+(k_{division}-k_{departure}) n_g(t,x) + k_{b\rightarrow g } n_{b}(t,x) $$

$$ \frac{d n_r (t,x)}{dt} = k_{entry}n_r(t,x_{neighbors})+(k_{division}-k_{departure}) n_r(t,x) + k_{b\rightarrow r } n_{b}(t,x) $$

Once we include noise, numerically solving this set of differential equations should recapitulate our data. To simplify analysis, we may choose to approximate $x_{neighbors}$ by $x$ because we expect density to vary slowly over the scale of a small number of analysis areas. Additionally, we have assumed above that $k_{division}$ is the same for all three cell types, as well as rates of motion. This should hold because these cell types are very similar, and are not readily distinguishable without these tags. 

The rates $k$ above are the average rates; in practice, the number densities will be non-deterministic due to various sources of error including: 

- errors due to cell counting algorithm (stochasticity from the observation)
    - To process our data set, we use a black-box cell counting software from MIT's Broad Institute. While this software is very good, it is still not perfect (and is also not as good as the naked eye). It will make errors due to issues such as image quality and resolution (blurring cell outlines), a wide spread of cell sizes (this type of segmentation algorithm usually does better when cells are relatively uniform in size), and possible overlap between cells. Because this is a mixture of many types of uncertainties, each of which has an unknown distribution. A gaussian is the most conservative way of estimating error, so we will treat these errors as gaussian with a single, non-time-varying $\sigma$ and mean 0. To determine the value of $\sigma_{obs}$ we will manually count the cells in a small subset of our images, then compare our counts against the actual count to determine the error of the computer algorithm. 
    
- stochasticity of cell behavior
    - In this particular case, researchers have observed that the Sox2 reporter is unstable--some cells will silence their fluorescently tagged Sox2 while maintaining their untagged copy.  We expect this $k_{silence}$ to be significantly smaller than the other values, but we should nonetheless include it. However, it is not clear what would be the distribution of uncertainties originated from this Poisson loss process of cell reporters. Including this uncertainties, the model will present statistical noise, and to be mostly conservative, we may assume the uncertainty to be Gaussian with variance $\sigma_{cells}^2$. The estimation of this variance requires further discussion.
    
$$ \frac{d n_b^* (t,x)}{dt} = \frac{d n_b (t,x)}{dt}-k_{silence}  n_b(t,x) $$ 
$$ \frac{d n_g^* (t,x)}{dt} = \frac{d n_g (t,x)}{dt}-k_{silence} n_g(t,x) $$
$$ \frac{d n_r^* (t,x)}{dt} = \frac{d n_r (t,x)}{dt}+k_{silence} n_b(t,x) $$




Let $\vec{n} = (n_b,n_g,n_r)$ (evaluated at any (x,t). Let $\vec{k}$ denote the parameter vector for rates $(k_{entry}, k_{departure}, k_{division}, k_{b \rightarrow g}, k_{b \rightarrow r}, k_{silence})$. We will numerically solve the above system to find $m(\vec{n}|\vec{k})$. Let $\vec{e }$ denote the gaussian observation error in $\vec{n}$ and let $\vec{\epsilon}$ denote the gaussian measurement observation error in $\vec{n}$ (as described in Gregory 4.8.2).

Then the prediction $\vec{\mathfrak{n}}$ from our generative model is:

$$\vec{z} = m(\vec{\mathfrak{n}}|\vec{k}) + \vec{e_i} + \vec{\epsilon_i}$$

We can then show, as Gregory equation 4.5:

$$p ( N| M, \vec{k},I) =\frac{1}{\sqrt{2 \pi (\sigma^2_{obs}+\sigma^2_{cells}}) } \exp \{ \frac{-(\vec{N}-\vec{m}(\vec{n}|\vec{k}))^2}{2(\sigma_{obs}^2+\sigma_{cells}^2)} \}$$

(Gregory 4.51 essentially)

Next we need to get from a single point in (x,t) to the full three dimensions (x,y,t) of our data. We assume that observational noise is independent (so will be separable). Noise inherent to our system is the sum of noise from many biological processes on different time-scales. We will assume this noise is independent our data points can still be treated as independent. We thus can multiply the likelihood of each data point to get the final likelihood function for our generative model. Let $T$ by the total number of time steps and $B$ be the total number of bins per time step.

$$p(D|M, \vec{k},I)=(2\pi)^{-B T /2} \{ \prod_{i=1}^T\prod_{j=1}^{B} (\sigma_{obs}^2 +\sigma_{cells}^2)^{-1/2} \} \exp \{ \sum_{i,j=1}^{i=T,j=B} (\frac{-(\vec{n_{i,j}}-\vec{m}(\vec{n_{i,j}}|\vec{k}))^2}{2(\sigma_{obs}^2+\sigma_{cells}^2)} ) \}$$

In [11]:
% matplotlib inline

import numpy as np
import pandas as pd
from cellfate import model, cell_density_object as cdo


params1 = model.test_params
params2 = [0, 0.05, 0, 0.6, 0.2, 0]
testobject = cdo.CellDen(pd.read_pickle('cellfate/sample.pkl'), 5)
a = model.log_likelihood(params1, testobject, 0.4)
b = model.log_likelihood(params2, testobject, 0.4)

odds = np.exp(a)/np.exp(b)

print(1/(1+1/odds))

1.0


In [16]:
model.np2pd(a)

R                                       G                       \
            0         1         2         3         0         1          2   
0    2.000000  3.000000  2.000000  1.000000  1.000000  2.000000   4.000000   
1    2.008048  3.019914  2.023732  1.004024  1.004024  2.055100   4.110200   
2    2.016129  3.039510  2.046762  1.008064  1.008064  2.108028   4.216056   
3    2.024242  3.058806  2.069127  1.012121  1.012121  2.158897   4.317795   
4    2.032388  3.077820  2.090864  1.016194  1.016194  2.207815   4.415630   
5    2.040567  3.096569  2.112006  1.020283  1.020283  2.254883   4.509767   
6    2.048778  3.115071  2.132585  1.024389  1.024389  2.300199   4.600398   
7    2.057023  3.133339  2.152633  1.028511  1.028511  2.343853   4.687706   
8    2.065300  3.151389  2.172178  1.032650  1.032650  2.385933   4.771866   
9    2.073611  3.169236  2.191248  1.036806  1.036806  2.426521   4.853043   
10   2.081956  3.186891  2.209870  1.040978  1.040978  2.465697   4.931394   
11   2.090334  3.204368  2.228067  1.045167  1.045167  2.503534   5.007068   
12   2.098746  3.221678  2.245865  1.049373  1.049373  2.540103   5.080207   
13   2.107191  3.238834  2.263285  1.053596  1.053596  2.575472   5.150944   
14   2.115671  3.255845  2.280349  1.057836  1.057836  2.609704   5.219408   
15   2.124185  3.272723  2.297076  1.062092  1.062092  2.642859   5.285719   
16   2.132733  3.289476  2.313487  1.066366  1.066366  2.674996   5.349992   
17   2.141315  3.306115  2.329599  1.070658  1.070658  2.706168   5.412336   
18   2.149932  3.322648  2.345431  1.074966  1.074966  2.736427   5.472854   
19   2.158584  3.339082  2.360997  1.079292  1.079292  2.765823   5.531646   
20   2.167270  3.355427  2.376314  1.083635  1.083635  2.794402   5.588804   
21   2.175992  3.371690  2.391397  1.087996  1.087996  2.822208   5.644417   
22   2.184748  3.387878  2.406260  1.092374  1.092374  2.849284   5.698568   
23   2.193540  3.403998  2.420916  1.096770  1.096770  2.875669   5.751339   
24   2.202367  3.420056  2.435379  1.101184  1.101184  2.901402   5.802803   
25   2.211230  3.436059  2.449659  1.105615  1.105615  2.926517   5.853035   
26   2.220128  3.452012  2.463769  1.110064  1.110064  2.951050   5.902101   
27   2.229062  3.467922  2.477719  1.114531  1.114531  2.975033   5.950066   
28   2.238032  3.483792  2.491520  1.119016  1.119016  2.998497   5.996993   
29   2.247038  3.499629  2.505182  1.123519  1.123519  3.021470   6.042940   
..        ...       ...       ...       ...       ...       ...        ...   
220  4.838871  7.603938  5.530135  2.419435  2.419435  6.912663  13.825327   
221  4.858343  7.634537  5.552389  2.429171  2.429171  6.940481  13.880963   
222  4.877893  7.665260  5.574733  2.438947  2.438947  6.968411  13.936822   
223  4.897523  7.696106  5.597166  2.448761  2.448761  6.996453  13.992907   
224  4.917231  7.727076  5.619690  2.458616  2.458616  7.024609  14.049217   
225  4.937019  7.758171  5.642305  2.468509  2.468509  7.052877  14.105754   
226  4.956886  7.789391  5.665010  2.478443  2.478443  7.081259  14.162518   
227  4.976833  7.820737  5.687807  2.488417  2.488417  7.109755  14.219511   
228  4.996861  7.852209  5.710696  2.498430  2.498430  7.138366  14.276733   
229  5.016969  7.883807  5.733677  2.508484  2.508484  7.167093  14.334185   
230  5.037158  7.915533  5.756750  2.518579  2.518579  7.195934  14.391868   
231  5.057428  7.947386  5.779916  2.528714  2.528714  7.224892  14.449784   
232  5.077780  7.979367  5.803175  2.538890  2.538890  7.253966  14.507932   
233  5.098213  8.011477  5.826528  2.549107  2.549107  7.283157  14.566315   
234  5.118729  8.043717  5.849975  2.559365  2.559365  7.312466  14.624932   
235  5.139328  8.076086  5.873516  2.569664  2.569664  7.341893  14.683785   
236  5.160009  8.108585  5.897152  2.580005  2.580005  7.371438  14.742876   
237  5.180774  8.141215  5.920883  2.590387  2.590387  7.401102  14.802203   
238  5.201622  8.173977  5.944710 